In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
import seaborn as sns
from rasterio.plot import show
import copy
import scipy.signal as signal
from scipy.spatial import ckdtree
from sklearn.cluster import DBSCAN
import circle_method_auxilliary as aux

In [2]:
#DATA 
raster_name = 'roads123_manualfilter'
src = rasterio.open('rasterdata/roads123_manualfilter.tif')
print(f'Raster imported - Raster size: {src.width} x {src.height} - Raster CRS: {src.crs}')

Raster imported - Raster size: 1268 x 1105 - Raster CRS: EPSG:4326


In [3]:
# RASTER processing
band = src.read(1)
raster_array = copy.deepcopy(band)
raster_array = np.where(raster_array != 1, 0, raster_array) # 0 = no road, 1 = road

road_indices = np.where(raster_array == 1)
row, col = road_indices[0], road_indices[1]
x,y  = src.xy(row, col) # x,y coordinates of road pixels
x_y = np.array([x,y]).T  

xy_df = pd.DataFrame(x_y, columns=['x', 'y'])
xy_df = aux.wgs84_to_web_mercator(xy_df)

print(f'# of road pixels: {len(xy_df)}')

# of road pixels: 48233


In [4]:
# KDTREE processing
tree = ckdtree.cKDTree((xy_df[['x', 'y']].values))
print(f'Number of points in tree: {len(tree.data)}')

neighbors = tree.query_ball_point(xy_df, r=100) # find neighbors within 100m

neighbors_df = pd.DataFrame(neighbors, columns=['neighbors'])
neighbors_df['neighbors_count'] = neighbors_df['neighbors'].apply(lambda x: len(x))

intersection_points = []
central_points = []

i = 0
print(f'Number of points: {len(x_y)}')
for i in range(0,len(neighbors)):
    if i % 5000 == 0: # print progress
        print(f'Point {i} of {len(neighbors)}')
    if len(neighbors[i]) < 10: 
        continue
    else:
        neighbors_points = xy_df.iloc[neighbors[i]]
        central_point = x_y[i][0], x_y[i][1]
        peak_amount, peaks = aux.count_peaks_simplified_angles(central_point, neighbors_points) 
        if peak_amount > 2: # if there are more than 2 peaks
            intersection_points.append(x_y[i])
            central_points.append(central_point)

print(f'Number of intersection points: {len(intersection_points)}')


Number of points in tree: 48233
Number of points: 48233
Point 0 of 48233
Point 5000 of 48233
Point 10000 of 48233
Point 15000 of 48233
Point 20000 of 48233
Point 25000 of 48233
Point 30000 of 48233
Point 35000 of 48233
Point 40000 of 48233
Point 45000 of 48233
Number of intersection points: 14207


In [5]:
# DBSCAN processing
eps = 30
min_samples = 7
db = DBSCAN(eps=eps, min_samples=min_samples).fit(intersection_points)
labels = db.labels_
print(f'Number of clusters: {len(set(labels)) - (1 if -1 in labels else 0)}')

# Remove outliers/noise
cluster_df = pd.DataFrame(intersection_points, columns=['x', 'y'])
cluster_df['cluster'] = labels
cluster_df = cluster_df[cluster_df['cluster'] != -1]


Number of clusters: 418


In [6]:
# CLuster processing

# Remove large clusters
max_treshold = cluster_df.groupby('cluster').count().quantile(0.95).values[0]
print(f'Max treshold: {max_treshold}')
large_clusters = cluster_df.groupby('cluster').count().where(lambda x: x['x'] > max_treshold).dropna().index

large_cluster_df = cluster_df[cluster_df['cluster'].isin(large_clusters)]

cluster_df_small = cluster_df[~cluster_df['cluster'].isin(large_clusters)]
cluster_df_small = cluster_df_small[cluster_df_small['cluster'] != -1]

# Calculate cluster centroids and find neighbors
cluster_centroid = cluster_df_small.groupby('cluster').mean()
centroid_neighbors = tree.query_ball_point(cluster_centroid, r=100)
centroid_neighbors_df = pd.DataFrame(centroid_neighbors, columns=['neighbors'])

Max treshold: 38.0


In [7]:
result_df = pd.DataFrame(columns=['x', 'y', 'cluster', 'peak_amount', 'peak_angles'])
result_df['peak_amount'] = result_df['peak_amount'].astype('object')
result_df['peak_angles'] = result_df['peak_angles'].astype('object')

# Find central points of clusters
for i in range(len(cluster_centroid)):
# for i in range(10):
    print(f'Iteration {i}')
    if i % 25 == 0: # print progress
        print(f'Point {i} of {len(cluster_centroid)}')
    current_cluster = cluster_centroid.iloc[i].copy()
    current_cluster_neighbors = xy_df.iloc[centroid_neighbors_df.iloc[i]['neighbors']]
    current_cluster_neighbors = current_cluster_neighbors.reset_index()

    x_max = max(current_cluster_neighbors['x'])
    x_min = min(current_cluster_neighbors['x'])
    y_max = max(current_cluster_neighbors['y'])
    y_min = min(current_cluster_neighbors['y'])

    x_arr = []
    y_arr = []

    for n in range(5):
        model1, model2 = aux.RANSAC_line(current_cluster_neighbors)
        x, y = aux.find_intersection(model1, model2)
        x_arr.append(x)
        y_arr.append(y)

    if np.isin(None, x_arr).any():
        x_intersection = np.nan
        y_intersection = np.nan
    else:
        x_intersection = np.mean(x_arr)
        y_intersection = np.mean(y_arr)


    radius = (x_max-x_min)*0.15
    distance_mean_intersection = np.sqrt((current_cluster['x'] - x_intersection)**2 + (current_cluster['y'] - y_intersection)**2)
    if x_intersection == np.nan or y_intersection == np.nan or distance_mean_intersection > (x_max-x_min)*0.4:
        circle_x, circle_y, centers = aux.iterative_circle_centering(current_cluster_neighbors, current_cluster['x'], current_cluster['y'], radius=radius*2)
    else:
        circle_x, circle_y, centers = aux.iterative_circle_centering(current_cluster_neighbors, np.mean(x_arr), np.mean(y_arr), radius=radius*2)

    # print(f'Cluster {i} - x: {circle_x} - y: {circle_y} --- original x: {current_cluster["x"]} - original y: {current_cluster["y"]}')

    result_df = result_df.append({'x': circle_x, 'y': circle_y, 'cluster': i}, ignore_index=True)

Iteration 0
Point 0 of 397
Cluster 0 - x: 13525052.260371702 - y: 3678716.56220567 --- original x: 13525049.245468829 - original y: 3678723.3515363145
Iteration 1
Cluster 1 - x: 13523900.72208361 - y: 3678541.309115904 --- original x: 13523895.279797392 - original y: 3678548.840616079
Iteration 2
Cluster 2 - x: 13525093.077518329 - y: 3678499.3066823203 --- original x: 13525091.8406351 - original y: 3678501.4792372305
Iteration 3
Cluster 3 - x: 13523707.149857953 - y: 3678440.648343519 --- original x: 13523698.654423129 - original y: 3678433.4447211614
Iteration 4
Cluster 4 - x: 13520601.33606482 - y: 3678362.437669674 --- original x: 13520590.999254959 - original y: 3678358.403033305
Iteration 5
Cluster 5 - x: 13523236.825009352 - y: 3678248.381330951 --- original x: 13523225.383839466 - original y: 3678238.60513532
Iteration 6
Cluster 6 - x: 13525831.496807257 - y: 3678099.566561463 --- original x: 13525869.67743963 - original y: 3678080.4718264216
Iteration 7
Cluster 7 - x: 13522833

In [8]:
result_df['peak_amount'] = result_df['peak_amount'].astype('object')
result_df['peak_angles'] = result_df['peak_angles'].astype('object')

# Find peaks and angles of clusters
for n in range(len(cluster_centroid)):
# for n in range(10):
    if n % 25 == 0: # print progress
        print(f'Point {n} of {len(cluster_centroid)}')
        
    current_cluster_x = result_df.iloc[n]['x']
    current_cluster_y = result_df.iloc[n]['y']
    current_cluster_neighbors = xy_df.iloc[centroid_neighbors_df.iloc[n]['neighbors']].copy()
    # current_cluster_neighbors['index'] = range(len(current_cluster_neighbors))

    print(f'Cluster {n} - x: {current_cluster_x} - y: {current_cluster_y}')
    print(f'Cluster {n} has {len(current_cluster_neighbors)} neighbors')

    peak_amount, peaks, histogram, edges = aux.count_peaks((current_cluster_x, current_cluster_y), current_cluster_neighbors, return_histogram=True)
    # peaks = np.where(peaks == 20, 0, peaks)
    peaks = peaks-1

    # print(f'Cluster {n} has {peak_amount} peaks')
    # print(f'Cluster {n} has peaks at {peaks}')

    peak_angles = aux.calculate_peak_angle(peaks, histogram)

    # print(f'Cluster {n} has peak angles at {peak_angles}')

    result_df['peak_amount'].iloc[n]= peak_amount
    result_df['peak_angles'].iloc[n] = peak_angles


Point 0 of 397
Cluster 0 - x: 13525052.260371702 - y: 3678716.56220567
Cluster 0 has 31 neighbors
Cluster 0 has 3 peaks
Cluster 0 has peaks at [ 3  9 18]
Cluster 0 has peak angles at [54.0, 166.5, 320.73]
Cluster 1 - x: 13523900.72208361 - y: 3678541.309115904
Cluster 1 has 32 neighbors


C:\Users\hokso\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Cluster 1 has 3 peaks
Cluster 1 has peaks at [ 4 13 19]
Cluster 1 has peak angles at [72.0, 234.0, 334.0]
Cluster 2 - x: 13525093.077518329 - y: 3678499.3066823203
Cluster 2 has 34 neighbors
Cluster 2 has 5 peaks
Cluster 2 has peaks at [ 0  4 10 15 19]
Cluster 2 has peak angles at [348.0, 72.0, 180.0, 270.0, 348.0]
Cluster 3 - x: 13523707.149857953 - y: 3678440.648343519
Cluster 3 has 34 neighbors
Cluster 3 has 3 peaks
Cluster 3 has peaks at [ 3  9 14]
Cluster 3 has peak angles at [50.4, 158.0, 244.8]
Cluster 4 - x: 13520601.33606482 - y: 3678362.437669674
Cluster 4 has 34 neighbors
Cluster 4 has 3 peaks
Cluster 4 has peaks at [ 8 13 19]
Cluster 4 has peak angles at [140.0, 229.5, 340.2]
Cluster 5 - x: 13523236.825009352 - y: 3678248.381330951
Cluster 5 has 32 neighbors
Cluster 5 has 3 peaks
Cluster 5 has peaks at [ 3  8 13]
Cluster 5 has peak angles at [49.5, 148.0, 243.0]
Cluster 6 - x: 13525831.496807257 - y: 3678099.566561463
Cluster 6 has 44 neighbors
Cluster 6 has 3 peaks
Cluster

In [10]:
csv_path = 'intersections'
csv_path = csv_path + '_raster_' + str(raster_name)
csv_path = csv_path + '_radius_' + str(100)
csv_path = csv_path + '_eps_' + str(eps)
csv_path = csv_path + '_minsamples_' + str(min_samples)
csv_path = csv_path + '.csv'


result_df.to_csv(csv_path)